In [9]:
# -*- coding: utf-8 -*-
import argparse
import os
import string
import sys
os.chdir('/data/hyeryung/mucoco')
from itertools import repeat
import torch.multiprocessing as mp
from typing import List

import numpy as np
import pandas as pd
import torch
import transformers
from datasets import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM, AutoTokenizer, AutoConfig

from new_module.utils.robertacustom import RobertaCustomForSequenceClassification
from new_module.locate.new_locate_utils import *
import new_module.losses as lossbuilder

import logging
import os
from typing import List

import torch
import torch.nn.functional as F
import transformers

In [2]:

logging.basicConfig(level=os.environ.get('LOGGING_LEVEL', 'DEBUG').upper(), 
                    format='%(message)s')
logger = logging.getLogger(__name__)


In [3]:
closs_weight = 0.5

In [4]:
def unravel(outputs_df):
    outputs_df=outputs_df.explode('generations',ignore_index=True)
    outputs_df['prompt']=outputs_df['prompt'].apply(lambda x: x['text'])
    outputs_df['generations']=outputs_df['generations'].apply(lambda x: x['text'] if isinstance(x, dict) else x)
    outputs_df = outputs_df.dropna().reset_index(drop=True)
    return outputs_df
toxic_gen=pd.read_json('new_module/data/toxicity-avoidance/dev_set.jsonl',lines=True)
toxic_gen=unravel(toxic_gen)
prompt = '“I went to use the bathroom, and I went in and Lance was bent'
prediction = toxic_gen.loc[toxic_gen['prompt']==prompt,'generations'].tolist()
ckpt_path = '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'
model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path)
device='cuda'
model = model.to(device)
loc_machine=LocateMachine(model,tokenizer)
res = loc_machine.locate_main(prediction, "attention", max_num_tokens = 6, unit="word", num_layer=10, label_id=0)

In [5]:
mlm_tokenizer = AutoTokenizer.from_pretrained('roberta-base',cache_dir='/data/hyeryung/hf_cache')
mlm = AutoModelForMaskedLM.from_pretrained('roberta-base',cache_dir='/data/hyeryung/hf_cache')
mlm = mlm.to(device)

Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0


In [6]:
masked_text = res

In [7]:
## replace tokens at the indices with mask tokens
inputs = mlm_tokenizer(
    masked_text, return_tensors="pt", padding=True, truncation=True
)
inputs = inputs.to(device) 
# inputs = mlm_tokenizer(
#     source_text + ' ' + masked_text[0], return_tensors="pt", add_special_tokens=False
# )

## make predictions for the masked indices
with torch.no_grad():
    logits = mlm(**inputs).logits

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
logits[:, :, special_token_ids] = -float("inf")

indices_in_mlm_tokens = (
    inputs.input_ids == mlm_tokenizer.mask_token_id
).nonzero(as_tuple=True)
# print(f"indices_in_mlm_tokens: {indices_in_mlm_tokens}")
## get top k tokens for each index

predicted_token_ids = torch.topk(
    logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
    k=10,
    dim=-1,
)
# print(f"predicted_token_ids: {predicted_token_ids}")
# print(f"mlm_tokenizer.batch_decode(predicted_token_ids.indices): {mlm_tokenizer.batch_decode(predicted_token_ids.indices)}")


In [13]:
config={#'model_paths':['gpt2-large','/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-energy-training/step_600_best_checkpoint'],
        # 'tokenizer_paths':['gpt2-large','/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-energy-training/step_600_best_checkpoint'],
        'model_paths':['gpt2-large','/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'],
        'tokenizer_paths':['gpt2-large','/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint/'],
        'model_types': ["AutoModelForCausalLM", "AutoModelForSequenceClassification"],
        'cache_dir': "/data/hyeryung/hf_cache",
        'target_type': "embeds",
        'method': "mlm-beamsearch-v0",
       'losses': ["gpt2", "classification_no_prefix_logprobloss"],
       'target_label_ids': [0,0] ,
       'build_loss_dict': {"coeff_steps": 200, "coeff_pattern": "constant", "loss_type": "xentropy", "length_normalize": False, "AR_temperature": 1.0, "AR_top_k": 0, "AR_top_p": 0.96, "max_output_length": 20},
       'min_epsilons': [0.75],
       'source_data': 'new_module/data/toxicity-avoidance/testset_gpt2_2500.jsonl',
       'locate_unit': 'word',
       'locate_method': 'grad_norm',
       'device': 'cuda',
       'k_per_location': 3,
       'closs_weight': 0.9}

In [11]:
masked_sequence = inputs

In [11]:
class dummyArgs:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

build_loss_args = dummyArgs(**config["build_loss_dict"])

In [14]:
## load tokenizer, models, define losses
name2tokenizer = {}
name2model = {}
name2config = {}
loss2tokenizer = {}
embed_luts = []

for i, model_path in enumerate(config["model_paths"]):
    if (
        model_path not in name2model
    ):  # making sure we are not loading the model twice in case some constraints use the same model.
        try:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=True,
            )
        except:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=False,
            )

        name2config[model_path] = AutoConfig.from_pretrained(
            model_path, cache_dir=config["cache_dir"]
        )

        if config["model_types"][i] == "RobertaCustomForSequenceClassification":
            name2model[model_path] = lossbuilder.ModelWrapper(
                RobertaCustomForSequenceClassification.from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        else:
            name2model[model_path] = lossbuilder.ModelWrapper(
                getattr(transformers, config["model_types"][i]).from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        name2model[model_path].eval()
        name2model[model_path].cuda()

    input_embeds = name2model[model_path].get_input_embeddings()
    if isinstance(input_embeds, torch.nn.Sequential):
        input_embeds = input_embeds[0]
    embed_luts.append(input_embeds)

    if config["target_type"] == "embeds":
        embed_luts[-1].requires_grad = False

mlm_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
mlm = None if config["method"] == "mlm-beamsearch-v2" else AutoModelForMaskedLM.from_pretrained("roberta-base").cuda()

lossfns = []
for i, loss in enumerate(config["losses"]):
    lossfns.append(
        lossbuilder.build_loss(
            loss,
            name2model[config["model_paths"][i]],
            name2tokenizer[config["tokenizer_paths"][i]],
            build_loss_args,
        )
    )
    lossfns[i].tokenizer.add_special_tokens({"mask_token": mlm_tokenizer.mask_token})
    loss2tokenizer[loss] = lossfns[i].tokenizer

https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Attempting to acquire lock 140090587200192 on /data/hyeryung/hf_cache/.locks/models--gpt2-large/be4d21d94f3b4687e5a54d84bf6ab46ed0f8defd.lock
Lock 140090587200192 acquired on /data/hyeryung/hf_cache/.locks/models--gpt2-large/be4d21d94f3b4687e5a54d84bf6ab46ed0f8defd.lock
https://huggingface.co:443 "GET /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 200 26


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Attempting to release lock 140090587200192 on /data/hyeryung/hf_cache/.locks/models--gpt2-large/be4d21d94f3b4687e5a54d84bf6ab46ed0f8defd.lock
Lock 140090587200192 released on /data/hyeryung/hf_cache/.locks/models--gpt2-large/be4d21d94f3b4687e5a54d84bf6ab46ed0f8defd.lock
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/vocab.json HTTP/1.1" 200 0
Attempting to acquire lock 140095390826560 on /data/hyeryung/hf_cache/.locks/models--gpt2-large/1f1d9aaca301414e7f6c9396df506798ff4eb9a6.lock
Lock 140095390826560 acquired on /data/hyeryung/hf_cache/.locks/models--gpt2-large/1f1d9aaca301414e7f6c9396df506798ff4eb9a6.lock
Attempting to release lock 140095390826560 on /data/hyeryung/hf_cache/.locks/models--gpt2-large/1f1d9aaca301414e7f6c9396df506798ff4eb9a6.lock
Lock 140095390826560 released on /data/hyeryung/hf_cache/.locks/models--gpt2-large/1f1d9aaca301414e7f6c9396df506798ff4eb9a6.lock
Starting new HTTPS connection (2): huggingface.co:443
Starting new HTTPS connection (3): huggingface.c

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Attempting to release lock 140090375955024 on /data/hyeryung/hf_cache/.locks/models--gpt2-large/226b0752cac7789c48f0cb3ec53eda48b7be36cc.lock
Lock 140090375955024 released on /data/hyeryung/hf_cache/.locks/models--gpt2-large/226b0752cac7789c48f0cb3ec53eda48b7be36cc.lock
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/added_tokens.json HTTP/1.1" 404 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/special_tokens_map.json HTTP/1.1" 404 0


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
hypotheses = [torch.LongTensor([]).to(config['device'])]
L = masked_sequence.size(-1)

for i in range(L):
    if masked_sequence[0, i] != mlm_tokenizer.mask_token_id:
        hypotheses = list(torch.cat([torch.stack(hypotheses,dim=0), 
                                    masked_sequence[:, i].unsqueeze(0).repeat((len(hypotheses),1)).to(config['device'])],dim=-1))
    else:
        num_hypotheses = len(hypotheses)
        hypotheses = torch.stack(hypotheses,dim=0).unsqueeze(0)
        hypotheses = hypotheses.repeat(config['k_per_location'], 1, 1)
        candidates = predicted_token_ids.indices[torch.where(indices_in_mlm_tokens == i)[0], :].to(config['device']).T.unsqueeze(1)
        candidates = candidates.repeat(1, num_hypotheses, 1)
        hypotheses_exp = torch.cat([hypotheses, candidates], dim=-1)
        hypotheses_exp = hypotheses_exp.view(-1, hypotheses_exp.shape[-1])
        hypotheses_exp = list(hypotheses_exp)

        losses = []
        loss_weights = [1 - closs_weight, closs_weight]
        for hyp in hypotheses_exp:
            curr_loss = 0.0
            for lossid, lossname in enumerate(config["losses"]):
                with torch.no_grad():
                    lossvalue = lossfns[lossid].compute_gold_loss(
                        source_text, mlm_tokenizer.decode(hyp, skip_special_tokens=True),
                        label_id=config['target_label_ids'][lossid],
                    )
                curr_loss += loss_weights[lossid] * lossvalue.item()
            losses.append(curr_loss)

        hypotheses = sorted(zip(hypotheses_exp, losses), key=lambda x: x[1])[:beam_size]
        hypotheses = [x[0] for x in hypotheses]

In [ ]:

def beam_rerank_v0(source_text, ## text (too arbitrary?)
                    masked_sequence, ## in mlm tokenizer's tokens
                    indices_in_mlm_tokens,
                    predicted_token_ids,
                    mlm_tokenizer, 
                    lossfns,
                    config, 
                    beam_size = 5):
    
    hypotheses = [torch.LongTensor([]).to(config['device'])]
    L = masked_sequence.size(-1)

    for i in range(L):
        if masked_sequence[0, i] != mlm_tokenizer.mask_token_id:
            hypotheses = list(torch.cat([torch.stack(hypotheses,dim=0), 
                                        masked_sequence[:, i].unsqueeze(0).repeat((len(hypotheses),1)).to(config['device'])],dim=-1))
        else:
            num_hypotheses = len(hypotheses)
            hypotheses = torch.stack(hypotheses,dim=0).unsqueeze(0)
            hypotheses = hypotheses.repeat(config['k_per_location'], 1, 1)
            candidates = predicted_token_ids.indices[torch.where(indices_in_mlm_tokens == i)[0], :].to(config['device']).T.unsqueeze(1)
            candidates = candidates.repeat(1, num_hypotheses, 1)
            hypotheses_exp = torch.cat([hypotheses, candidates], dim=-1)
            hypotheses_exp = hypotheses_exp.view(-1, hypotheses_exp.shape[-1])
            hypotheses_exp = list(hypotheses_exp)

            losses = []
            loss_weights = [1 - wandb.config.closs_weight, wandb.config.closs_weight]
            for hyp in hypotheses_exp:
                curr_loss = 0.0
                for lossid, lossname in enumerate(config["losses"]):
                    with torch.no_grad():
                        lossvalue = lossfns[lossid].compute_gold_loss(
                            source_text, mlm_tokenizer.decode(hyp, skip_special_tokens=True),
                            label_id=config['target_label_ids'][lossid],
                        )
                    curr_loss += loss_weights[lossid] * lossvalue.item()
                losses.append(curr_loss)

            hypotheses = sorted(zip(hypotheses_exp, losses), key=lambda x: x[1])[:beam_size]
            hypotheses = [x[0] for x in hypotheses]
            
    return [mlm_tokenizer.decode(x, skip_special_tokens=True) for x in hypotheses]
